# Querying multiple APIs together in one experiment asynchoronously

In [ ]:
import time
import os
import requests
from dotenv import load_dotenv

from prompto.settings import Settings
from prompto.experiment import Experiment

from api_utils import send_prompts_sync
from dataset_utils import load_prompt_dicts, load_prompts, generate_experiment_1_file, combine_experiment_files

load_dotenv(dotenv_path=".env")

## Setup input jsonl files

In [ ]:
alpaca_prompts = load_prompts("./sample_prompts.json")[:10]

In [ ]:
OPENAI_EXPERIMENT_FILE = "./data/input/openai.jsonl"
GEMINI_EXPERIMENT_FILE = "./data/input/gemini.jsonl"
OLLAMA_EXPERIMENT_FILE = "./data/input/ollama.jsonl"
COMBINED_EXPERIMENT_FILENAME = "./data/input/all_experiments.jsonl"
INPUT_EXPERIMENT_FILEDIR = "./data/input"

if not os.path.isdir(INPUT_EXPERIMENT_FILEDIR):
    os.mkdir(INPUT_EXPERIMENT_FILEDIR)

Generate the three files seperate experiment json files. Then load them in and merge into a single file. 

In [ ]:
generate_experiment_1_file(
    path=OPENAI_EXPERIMENT_FILE,
    prompts=alpaca_prompts,
    api="openai",
    model_name="gpt-3.5-turbo",
    params={"n": 1, "temperature": 0.9, "max_tokens": 100},
)

generate_experiment_1_file(
    path=GEMINI_EXPERIMENT_FILE,
    prompts=alpaca_prompts,
    api="gemini",
    model_name="gemini-1.5-flash",
    params={"candidate_count": 1, "temperature": 0.9, "max_output_tokens": 100},
)

generate_experiment_1_file(
    path=OLLAMA_EXPERIMENT_FILE,
    prompts=alpaca_prompts,
    api="ollama",
    model_name="llama3",
    params={"temperature": 0.9, "num_predict": 100, "seed": 42},
)

In [ ]:
combine_experiment_files(experiment_paths=[OPENAI_EXPERIMENT_FILE, GEMINI_EXPERIMENT_FILE, OLLAMA_EXPERIMENT_FILE], output_path=COMBINED_EXPERIMENT_FILENAME)

In [ ]:
print(
    f"len(load_prompt_dicts(COMBINED_EXPERIMENT_FILENAME)): {len(load_prompt_dicts(COMBINED_EXPERIMENT_FILENAME))}"
)

In [ ]:
multiple_api_experiment = Experiment(
    file_name="all_experiments.jsonl", settings=Settings(data_folder="./data", max_queries=500)
)

start = time.time()
openai_responses, _ = await multiple_api_experiment.process()
print(time.time() - start)